In [ ]:
import requests

session = requests.sessions.Session()

In [ ]:
from io import StringIO
import pandas as pd

term = "202503"

main_df = pd.DataFrame()
count = 0
while True:
    response = session.get(f"https://duckweb.uoregon.edu/duckweb/hwskdhnt.P_ListCrse?term_in={term}&sel_subj=dummy&sel_day=dummy&sel_schd=dummy&sel_insm=dummy&sel_camp=dummy&sel_levl=dummy&sel_sess=dummy&sel_instr=dummy&sel_ptrm=dummy&sel_attr=dummy&sel_cred=dummy&sel_tuition=dummy&sel_open=dummy&sel_weekend=dummy&sel_title=&sel_to_cred=&sel_from_cred=&sel_subj=%25&sel_crse=&sel_crn=&begin_hh=0&begin_mi=0&begin_ap=a&end_hh=0&end_mi=0&end_ap=a&submit_btn=Summary&cidx={count}")
    if 'No classes were found that meet your search criteria' in response.text:
        break
    with StringIO(response.text) as buffer:
        df_group = pd.read_html(buffer)

    df = [x for x in df_group if x.iloc[0, 0] == 'Subj'][0].iloc[:-3]
    df.columns = df.iloc[0]
    df = df.loc[:, df.columns.notna()]
    df = df[1:]
    main_df = pd.concat([main_df, df])
    count += 100
main_df

,Subj,Crse,Title,Creds,CRN,Avail,Max,Time,Day,Location,Instructor,Notes
1,AAAP,411,Intro Historic Preserv,3.00 cr.,30227,14,14,0900-1150,f,263 LA,Bell C,NaN
2,AAAP,411,Intro Historic Preserv,3.00 cr.,30227,14,14,0900-1150,f,263 LA,Rudnicki L,NaN
3,AAAP,511,Intro Historic Preserv,3.00 cr.,30228,14,16,0900-1150,f,263 LA,Bell C,NaN
4,AAAP,511,Intro Historic Preserv,3.00 cr.,30228,14,16,0900-1150,f,263 LA,Rudnicki L,NaN
5,AAAP,611,Terminal Project,1.00-12.00 cr.,30229,10,10,tba,tba,tba,STAFF,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
94,WR,123,College Composit III,4.00 cr.,35129,25,25,1400-1520,tr,NaN,Hatay M,!
95,WR,320,Scientific & Tech Wr,4.00 cr.,35135,22,22,-,NaN,NaN,Recker I,!$
96,WR,321,Business Communication,4.00 cr.,35136,22,22,1600-1720,mw,NaN,Smith B,!
97,WR,408,Lab Indpend Writ Proj,1.00-3.00 cr.,35137,5,5,tba,tba,tba,STAFF,NaN


In [21]:
main_df = main_df.drop_duplicates('CRN').reset_index(drop=True)

In [23]:
import sqlite3

conn = sqlite3.connect('classes.db')

main_df.to_sql('Spring2026Classes', conn, index=False)
conn.close()